In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import pandas as pd
import requests
import json
import re
from string import punctuation
from pymystem3 import Mystem
from stop_words import get_stop_words
from email.message import EmailMessage
from smtplib import SMTP
import smtplib, ssl
import gspread

gauth = GoogleAuth()
# Try to load saved client credentials
gauth.LoadCredentialsFile("mycreds.txt")
if gauth.credentials is None:
    # Authenticate if they're not there
    gauth.LocalWebserverAuth()
elif gauth.access_token_expired:
    # Refresh them if expired
    gauth.Refresh()
else:
    # Initialize the saved creds
    gauth.Authorize()
# Save the current credentials to a file
gauth.SaveCredentialsFile("mycreds.txt")
drive = GoogleDrive(gauth)

url = 'https://10education.ru'
r = requests.get(url)
first_list = r.text.split('<br />')
index_pass = first_list[0].index('/html')
first_list[0] = first_list[0][index_pass+10:]
second_list = [string.split(' - ') for string in first_list]
df = pd.DataFrame(second_list)
columns_names = {0: 'username', 1: 'email', 2: 'password', 3: 'history_new'}
data = df.rename(columns=columns_names)
data['email'] = data['email'].str.lower()
data['ma_email'] = data['email']
data = data[:-1]
print('Данные получены: '+url)

file = drive.ListFile({'q': "title = 'rules.xlsx'"}).GetList()[0]
file.GetContentFile(file['title'])
rules = pd.ExcelFile(file['title'])
rules = rules.parse('Концовки')

file = drive.ListFile({'q': "title = 'Теория технологического предпринимательства - тест.xlsx'"}).GetList()[0]
file.GetContentFile(file['title'])
test = pd.read_excel(file['title'])
test['E-mail'] = test['E-mail'].str.lower()

file = drive.ListFile({'q': "title = 'Cases'"}).GetList()[0]
file.GetContentFile(file['title']+'.xlsx', mimetype = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
cases = pd.read_excel(file['title']+'.xlsx', dtype = str, keep_default_na = False)
cases['ma_email'] = cases['ma_email'].str.lower()

Данные получены: https://10education.ru


In [2]:
def novella_check(email):
    if data[data['email'] == email]['history_new'].to_list() != []:
        h_str = data[data['email'] == email]['history_new'].to_list()[0]
        h_json = json.loads(h_str)
        h_list = [rec for rec in h_json.values() if rec != {}]
        if h_list == []:
            return(rules['Новелла'][1])
        h_record = max(h_list, key=max)
        if 'ending' in h_record.keys():
            return(rules[rules['Концовка']==h_record['ending']]['Новелла'].values[0])
        else:
            return(rules['Новелла'][1])
    else:
        return(rules['Новелла'][0])
    
def test_check(email):
    if email in test['E-mail'].values:
        pts = test[test['E-mail']==email]['score'].max()
        if pts < 50:
            return("Вы набрали менее половины из возможных баллов. В этом нет ничего страшного - корректные ответы на вопросы требуют подробного ознакомления с приложенными к практикуму материалами или большого личного опыта в сфере технологического предпринимательства.\n"
            "Если подобный результат вас не устраивает или ваш интерес к технологическому предпринимательству проснулся только после завершения практикума, то учиться никогда не поздно! Мы рекомендуем прослушать курс лекций П.Г. Щедровицкого о технологическом предпринимательстве, доступный в свободном доступе на ютубе. Отличное место чтобы начать - лекция \"Понятие предпринимательства\".")
        elif pts <= 75:
            return("Вы набрали более половины баллов - это неплохой результат!\n"
            "Вопросы данного теста во многом полагаются на знание теории технологического предпринимательства, и если эта тема вас интересует, то мы рекомендуем углубить свои знания, прослушав курс лекций П.Г. Щедровицкого о технологическом предпринимательстве, доступный в свободном доступе на ютубе. Отличное место чтобы начать - лекция \"Понятие предпринимательства\".")
        elif pts > 75:
            return("Однако, вам эти комментарии не понадобились - вы отлично справились с тестом!\n"
            "Однако мы рекомендуем не довольствоваться достигнутым, а продолжать углублять свои знания теории технологического предпринимательства при каждой удобной возможности. Так, например, мы рекомендуем ознакомиться с лекцией П.Г. Щедровицкого \"Индустриализация России\" доступной в свободном доступе на ютубе.")
    else:
        return("Согласно нашим данным, вы не приступали к прохождению данного этапа.\nЕсли вы считаете, что это ошибка - свяжитесь с нами!")
    
# Lemmatizer and stopwords
mystem = Mystem() 
stop_words = set(get_stop_words('russian'))
primary_cases_list = ['case1', 'case2', 'case3', 'case4']
dict_cases = {
    'case1': "представлять предприниматель компания заниматься разработка высокоточный станок производство пробный партия необходимо купить специальный клей исследовать возможный предложение интернет понимать поставка организовывать германия необходимый количество стоять евро самостоятельно приходиться сделать следующий заплатить закупка доставка налог пошлина потратить рабочий оформление документ ждать счастие инженер отец жить близкий собираться навещать сын заказывать домой привозить багаж неделя действовать официальный воспользоваться удачный стечение обстоятельство написать цепочка рассуждение подробно аргументировать ответ".split(' '),
    'case2': "представлять коммерческий директор компания разработка отопительный котел новый поколение месяц выпускать прототип показывать заказчик огромный строительный фирма оставаться довольный сделать следующий предложение готовый купить экземпляр пробный запускать достраивать здание проба продлиться оказываться годный сразу партия знать рынок конкурент каков особенность продукция данный момент участвовать испытание соглашаться отдавать начинать оформлять тз производство штука контрактный главный разработчик сообщать находить решение повышать производительность переделывать опытный образец конструкторский бюро проверять тестовый стенд число поменять короб помещаться отправлять устаревать конструкция инженер говорить срочно предлагать вернуть тестирование утверждать  % ,  вариант причина выбирать стоять вопрос обращаться прерывать середина путь технический характеристика продукт рискнуть потратить деньги подготовка документация улучшать точно скоро хороший примета складываться ситуация написать цепочка рассуждение подробно аргументировать ответ".split(' '),
    'case3': "представлять молодой предприниматель открывать небольшой стартап партнерство одногруппник разрабатывать крутой приложение смартфон уверять взлетать пользоваться огромный спрос очередной этап разработка понимать свой деньги хватать нужный инвестиция нанимать дополнительный разработчик дизайнер тестировщик покупать лицензия облако место надежный сервер посчитать близкий находить млн рубль неделя поиск приходить кредитный предложение банк предлагать вполне сносный условие взять льготный годовой получать перевод завтра отвечать венчурный фонд mobileventures согласный проинвестировать транш входить доля ждать результат месяц возможный следующий раунд переговоры партнер встречаться двоюродный дядя рассказывать идея понравиться склон скопить приличный сумма заниматься поставка продукт питание из-за рубеж готовый вкладывать единственный поставлять держать курс дело принятие решение интересно следить строиться новый технология поступать источник вообще написать цепочка рассуждение подробно аргументировать ответ".split(' '),
    'case4': "представлять начальник департамент инновация крупный корпорация подобный ibm усиливать инновационность деятельность собственно продукт достаточно большой бюджет поставлять амбициозный задача ряд проектный сессия вариант деньги выделять хватить начинать инвестировать самостоятельно фонд молодой стартапа нужный направление ожидать выходить улучшать повышать эффективность работа самый инвестиция предполагать получение доля компания должный возвращать последствие либо полностью выкупать процент успешность оценивать предлагать сотрудник сумма экономия приносить идея обязывать гарантировать рассмотрение поступать обоснованный ответ случай отказ принимать  –  таков корпоративный этика решение совещание совет директор".split(' ')
    }

# Preprocess function
def preprocess_text(text):
    tokens = mystem.lemmatize(re.sub(r'\d+', '', text.lower()))
    tokens = [token for token in tokens if token not in stop_words
              and token != " "
              and token.strip() not in punctuation]
    return list(dict.fromkeys(tokens))

def case_counter(text, case):
    tokens = preprocess_text(text)
    return sum(word in dict_cases[case] for word in tokens)

def cases_check(email):
    pts = 0
    if email in cases['ma_email'].values:
        for key in primary_cases_list:
            data_case = cases[cases['ma_email']==email][key].max()
            if len(data_case) >= 50:
                pts += 1
            if len(data_case) >= 300:
                pts += 2
            counter = case_counter(data_case, key)
            if counter >= 4:
                pts += 2
            if counter >= 6:
                pts += 1
            if counter >= 8:
                pts += 1
    if pts == 0:
        return("Согласно нашим данным, вы не приступали к прохождению данного этапа.\nЕсли вы считаете, что это ошибка - свяжитесь с нами!")
    elif pts < 5:
        return("Вы сформулировали свои ответы очень кратко - даже если вы указали оптимальное решение для кейса, ваши ответы не дают понять хода ваших мыслей. Умение аргументировать свою позицию очень важно в предпринимательской деятельности - практикуйте его!")
    elif pts <= 20:
        return("Приведённые вами решения кейсов свидетельствуют о наличии здравого делового смысла и умении рассуждать и аргументировать свою позицию. Если вы займётесь или продолжите заниматься деятельностью технологического предпринимателя, то рано или поздно вы столкнётесь с тем, что во многих деловых ситуациях решение, кажущееся наиболее логичным с точки зрения \"здравого смысла\" не всегда явлется самым оптимальным. В этих случаях на помощь в поиске решения к вам придёт именно способность рассуждать логически, опираясь на понимание теории предпринимательской деятельности. Продолжайте развивать эти навыки, и ваши шансы построить успешный технологический бизнес возрастут!")
    elif pts > 20:
        return("Приведённые вами решения кейсов демонстрируют как хорошее понимание предпринимательской логики, так и умение рассуждать логически и аргументировать свою позицию - всё критически важные черты для успешного строительства технологического бизнеса. Если вы ещё не занимаетесь или не планируете заняться технологическим предпринимательством - вам стоит подумать об этом!")

In [3]:
def send_finish(email):
    name = finish[finish['ma_email']==email]['ma_name'].max()
    testresults, novelresults, caseresults = test_check(email), novella_check(email), cases_check(email)
    html = f"""
    <!DOCTYPE html><html xmlns:v="urn:schemas-microsoft-com:vml" xmlns:o="urn:schemas-microsoft-com:office:office" lang="en"><head><title></title><meta http-equiv="Content-Type" content="text/html; charset=utf-8"><meta name="viewport" content="width=device-width,initial-scale=1"><!--[if mso]><xml><o:OfficeDocumentSettings><o:PixelsPerInch>96</o:PixelsPerInch><o:AllowPNG/></o:OfficeDocumentSettings></xml><![endif]--><!--[if !mso]><!--><link href="https://fonts.googleapis.com/css?family=Open+Sans" 
    rel="stylesheet" type="text/css"><link href="https://fonts.googleapis.com/css?family=Montserrat" rel="stylesheet" type="text/css"><link href="https://fonts.googleapis.com/css?family=Ubuntu" rel="stylesheet" type="text/css"><!--<![endif]--><style>
    *{{box-sizing:border-box}}body{{margin:0;padding:0}}a[x-apple-data-detectors]{{color:inherit!important;text-decoration:inherit!important}}#MessageViewBody a{{color:inherit;text-decoration:none}}p{{line-height:inherit}}@media (max-width:640px){{.row-content{{width:100%!important}}.column .border{{display:none}}table{{table-layout:fixed!important}}.stack .column{{width:100%;display:block}}</style></head><body style="background-color:#fff;margin:0;padding:0;-webkit-text-size-adjust:none;text-size-adjust:none"><table
     class="nl-container" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff"><tbody><tr><td><table class="row row-1" align="center" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#2d2d2d"><tbody><tr><td><table class="row-content stack" align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" 
    style="mso-table-lspace:0;mso-table-rspace:0;background-color:#2d2d2d;color:#333;width:620px" width="620"><tbody><tr><td class="column column-1" width="100%" style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;padding-top:25px;padding-bottom:0;border-top:0;border-right:0;border-bottom:0;border-left:0"><table class="text_block" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" 
    style="mso-table-lspace:0;mso-table-rspace:0;word-break:break-word"><tr><td style="padding-left:10px;padding-right:10px;padding-top:10px"><div style="font-family:'Trebuchet MS',Tahoma,sans-serif"><div class="txtTinyMce-wrapper" style="font-size:12px;font-family:Montserrat,'Trebuchet MS','Lucida Grande','Lucida Sans Unicode','Lucida Sans',Tahoma,sans-serif;mso-line-height-alt:14.399999999999999px;color:#2d2d2d;line-height:1.2"><p style="margin:0">
    <span style="font-size:30px;color:#ffffff;"><strong><span style>Здрвствуйте, {name}!</span></strong></span></p></div></div></td></tr></table><table class="text_block" width="100%" border="0" cellpadding="10" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;word-break:break-word"><tr><td><div style="font-family:'Trebuchet MS',Tahoma,sans-serif"><div class="txtTinyMce-wrapper" 
    style="font-size:12px;mso-line-height-alt:14.399999999999999px;color:#000;line-height:1.2;font-family:Montserrat,'Trebuchet MS','Lucida Grande','Lucida Sans Unicode','Lucida Sans',Tahoma,sans-serif">

    </div></div></td></tr></table></td></tr></tbody></table></td></tr></tbody></table><table class="row row-2" 
    align="center" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff"><tbody><tr><td><table class="row-content stack" align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff;color:#000;width:620px" width="620"><tbody><tr><td class="column column-1" width="100%" 
    style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;padding-top:5px;padding-bottom:5px;border-top:0;border-right:0;border-bottom:0;border-left:0"><table class="text_block" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;word-break:break-word"><tr><td style="padding-bottom:10px;padding-left:10px;padding-right:10px;padding-top:30px"><div 
    style="font-family:'Trebuchet MS',Tahoma,sans-serif"><div class="txtTinyMce-wrapper" style="font-size:14px;font-family:Montserrat,'Trebuchet MS','Lucida Grande','Lucida Sans Unicode','Lucida Sans',Tahoma,sans-serif;mso-line-height-alt:16.8px;color:#555;line-height:1.2"><p style="margin:0;font-size:20px"><span style="color:#000000;">Благодарим за прохождение онлайн-практикума Школы предпринимательства ТехноСпарка. Мы рады работать </span></p><p style="margin:0;font-size:20px">
    <span style="color:#000000;">с теми, кто заинтересован в теории или практике технологического предпринимательства!</span></p><p style="margin:0;font-size:20px;mso-line-height-alt:16.8px">&nbsp;</p><p style="margin:0;font-size:20px"><span style="color:#000000;font-size:20px;">Наши эксперты проанализировали ваши результаты </span></p><p style="margin:0;font-size:20px"><span style="color:#000000;font-size:20px;">и составили для вас краткие комментарии </span></p><p style="margin:0;font-size:20px">
    <span style="color:#000000;font-size:20px;">по некоторым из этапов практикума.</span></p><p style="margin:0;font-size:20px;mso-line-height-alt:16.8px">&nbsp;</p><p style="margin:0;font-size:20px"><span style="color:#000000;font-size:20px;">Напоминаем, что все этапы —&nbsp; это отбор в программу #стартапдиплом —&nbsp; предпринимательскую практику, запущенную ТехноСпарком совместно с российскими университетами.</span></p></div></div></td></tr></table></td></tr></tbody></table></td></tr></tbody></table><table 
    class="row row-3" align="center" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff"><tbody><tr><td><table class="row-content stack" align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff;color:#000;width:620px" width="620"><tbody><tr><td class="column column-1" width="100%" 
    style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;padding-top:5px;padding-bottom:5px;border-top:0;border-right:0;border-bottom:0;border-left:0"><table class="text_block" width="100%" border="0" cellpadding="10" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;word-break:break-word"><tr><td><div style="font-family:'Trebuchet MS',Tahoma,sans-serif"><div class="txtTinyMce-wrapper" 
    style="font-size:14px;font-family:Montserrat,'Trebuchet MS','Lucida Grande','Lucida Sans Unicode','Lucida Sans',Tahoma,sans-serif;mso-line-height-alt:16.8px;color:#555;line-height:1.2"><p style="margin:0;font-size:20px;text-align:center"><span style="font-size:30px;"><strong><span style="color:#b91919;">Ваши результаты!</span></strong></span></p></div></div></td></tr></table></td></tr></tbody></table></td></tr></tbody></table><table class="row row-4" align="center" width="100%" border="0" 
    cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0"><tbody><tr><td><table class="row-content stack" align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;color:#000;width:620px" width="620"><tbody><tr><td class="column column-1" width="41.666666666666664%" 
    style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;border-top:0;border-right:0;border-bottom:0;border-left:0"><table class="image_block" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0"><tr><td style="width:100%;padding-right:0;padding-left:0;padding-top:5px;padding-bottom:5px"><div align="center" style="line-height:10px"><img 
    src="https://10education.ru/images/elearning.png" style="display:block;height:auto;border:0;width:258px;max-width:100%" width="258"></div></td></tr></table></td><td class="column column-2" width="58.333333333333336%" 
    style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;border-top:0;border-right:0;border-bottom:0;border-left:0"><table class="text_block" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;word-break:break-word"><tr><td style="padding-bottom:20px;padding-top:5px"><div style="font-family:'Trebuchet MS',Tahoma,sans-serif"><div class="txtTinyMce-wrapper" 
    style="font-size:14px;font-family:Montserrat,'Trebuchet MS','Lucida Grande','Lucida Sans Unicode','Lucida Sans',Tahoma,sans-serif;mso-line-height-alt:16.8px;color:#555;line-height:1.2"><p style="margin:0"><span style="color:#b91919;font-size:20px;"><strong>Тест</strong></span></p><p style="margin:0;mso-line-height-alt:16.8px">&nbsp;</p><p style="margin:0">
    <span style="color:#000000;">Данный тест призван оценить уровень знакомства участника практикума с основами теории технологического предпринимательства - и рассказать о ней там, где участнику </span></p><p style="margin:0"><span style="color:#000000;">не хватает информации.</span></p><p style="margin:0;mso-line-height-alt:16.8px">&nbsp;</p><p style="margin:0">
    <span style="color:#000000;">{testresults}&nbsp;</span></p></div></div></td></tr></table></td>
    </tr></tbody></table></td></tr></tbody></table><table class="row row-5" align="center" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0"><tbody><tr><td><table class="row-content stack" align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;color:#000;width:620px" width="620"><tbody><tr><td class="column column-1" width="58.333333333333336%" 
    style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;border-top:0;border-right:0;border-bottom:0;border-left:0"><table class="text_block" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;word-break:break-word"><tr><td style="padding-bottom:20px;padding-top:5px"><div style="font-family:'Trebuchet MS',Tahoma,sans-serif"><div class="txtTinyMce-wrapper" 
    style="font-size:14px;font-family:Montserrat,'Trebuchet MS','Lucida Grande','Lucida Sans Unicode','Lucida Sans',Tahoma,sans-serif;mso-line-height-alt:16.8px;color:#555;line-height:1.2"><p style="margin:0"><span style="color:#b91919;"><span style="font-size:20px;">Интерактивная новелла</span></span></p><p style="margin:0;mso-line-height-alt:16.8px">&nbsp;</p><p style="margin:0">
    <span style="color:#000000;">Целью данного этапа является дать участнику попробовать примерить на себя роль предпринимателя в ряде ситуаций, наиболее часто встречающихся в жизни технологических стартапов. Первый стартап начинающего технологического предпринимателя как правило губит одна из опасных ситуаций, которыми до края наполнена история Евгения.</span></p><p style="margin:0;mso-line-height-alt:16.8px">&nbsp;</p><p style="margin:0">
    <span style="color:#000000;">{novelresults}&nbsp;&nbsp;</span></p></div></div></td></tr></table></td><td class="column column-2" width="41.666666666666664%" style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;border-top:0;border-right:0;border-bottom:0;border-left:0"><table class="image_block" width="100%" border="0" cellpadding="0" cellspacing="0" 
    role="presentation" style="mso-table-lspace:0;mso-table-rspace:0"><tr><td style="width:100%;padding-right:0;padding-left:0;padding-top:5px;padding-bottom:5px"><div align="center" style="line-height:10px"><img src="https://10education.ru/images/слой2.png" style="display:block;height:auto;border:0;width:258px;max-width:100%" width="258"></div></td></tr></table>
    </td></tr></tbody></table></td></tr></tbody></table><table class="row row-6" align="center" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0"><tbody><tr><td><table class="row-content stack" align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;color:#000;width:620px" width="620"><tbody><tr><td class="column column-1" width="41.666666666666664%" 
    style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;border-top:0;border-right:0;border-bottom:0;border-left:0"><table class="image_block" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0"><tr><td style="width:100%;padding-right:0;padding-left:0;padding-top:5px;padding-bottom:5px"><div align="center" style="line-height:10px"><img 
    src="https://10education.ru/images/слой1.png" style="display:block;height:auto;border:0;width:258px;max-width:100%" width="258"></div></td></tr></table></td><td class="column column-2" width="58.333333333333336%" 
    style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;border-top:0;border-right:0;border-bottom:0;border-left:0"><table class="text_block" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;word-break:break-word"><tr><td style="padding-bottom:20px;padding-top:5px"><div style="font-family:'Trebuchet MS',Tahoma,sans-serif"><div class="txtTinyMce-wrapper" 
    style="font-size:14px;font-family:Montserrat,'Trebuchet MS','Lucida Grande','Lucida Sans Unicode','Lucida Sans',Tahoma,sans-serif;mso-line-height-alt:16.8px;color:#555;line-height:1.2"><p style="margin:0"><span style="color:#b91919;font-size:20px;">Кейсы</span></p><p style="margin:0;mso-line-height-alt:16.8px">&nbsp;</p><p style="margin:0"><span style="color:#000000;">Все приведённые в практикуме кейсы основаны на реальных ситуациях, случившихся </span></p><p style="margin:0">
    <span style="color:#000000;">с реальными технологическими предпринимателями, строящими бизнес </span></p><p style="margin:0"><span style="color:#000000;">в нашей стране. И, ровно как и в реальной жизни, на них нет однозначного "правильного" ответа. Вместо этого, каждый кейс - это приглашение продемонстрировать логику, которую вы используете при анализе и решении деловых ситуаций.</span></p><p style="margin:0;mso-line-height-alt:16.8px">&nbsp;</p><p style="margin:0">
    <span style="color:#000000;">{caseresults}</span></p><p style="margin:0"><span style="color:#000000;"></span></p><p style="margin:0">
    <span style="color:#000000;">&nbsp;</span></p></div></div></td></tr></table></td></tr></tbody></table></td></tr></tbody></table><table class="row row-7" align="center" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff"><tbody><tr><td><table class="row-content stack" align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" 
    style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff;color:#000;width:620px" width="620"><tbody><tr><td class="column column-1" width="75%" style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;border-top:0;border-right:0;border-bottom:0;border-left:0"><table class="text_block" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;word-break:break-word"><tr><td 
    style="padding-bottom:10px;padding-left:10px;padding-right:10px;padding-top:15px"><div style="font-family:'Trebuchet MS',Tahoma,sans-serif"><div class="txtTinyMce-wrapper" style="font-size:12px;mso-line-height-alt:14.399999999999999px;color:#555;line-height:1.2;font-family:Montserrat,'Trebuchet MS','Lucida Grande','Lucida Sans Unicode','Lucida Sans',Tahoma,sans-serif"><p style="margin:0;font-size:12px;mso-line-height-alt:14.399999999999999px">&nbsp;</p></div></div></td></tr></table><table 
    class="image_block" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0"><tr><td style="width:100%;padding-right:0;padding-left:0;padding-bottom:5px"><div align="center" style="line-height:10px"><img 
    src="https://10education.ru/images/изображение_2021-08-17_121638.png" style="display:block;height:auto;border:0;width:465px;max-width:100%" width="465"></div></td></tr></table></td><td class="column column-2" width="25%" 
    style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;border-top:0;border-right:0;border-bottom:0;border-left:0"><table class="empty_block" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0"><tr><td style="padding-right:0;padding-bottom:5px;padding-left:0;padding-top:5px"><div></div></td></tr></table></td></tr></tbody></table></td></tr></tbody></table><table class="row row-8" 
    align="center" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff"><tbody><tr><td><table class="row-content stack" align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff;color:#000;width:620px" width="620"><tbody><tr><td class="column column-1" width="50%" 
    style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;border-top:0;border-right:0;border-bottom:0;border-left:0"><table class="text_block" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;word-break:break-word"><tr><td style="padding-bottom:15px;padding-left:10px;padding-right:10px;padding-top:15px"><div style="font-family:'Trebuchet MS',Tahoma,sans-serif"><div 
    class="txtTinyMce-wrapper" style="font-size:14px;font-family:Montserrat,'Trebuchet MS','Lucida Grande','Lucida Sans Unicode','Lucida Sans',Tahoma,sans-serif;mso-line-height-alt:16.8px;color:#555;line-height:1.2"><p style="margin:0;font-size:28px"><span style="font-size:28px;">Программа <span style="color:#b91919;">#стартапдиплом</span></span></p><p style="margin:0;font-size:28px"><span style="font-size:28px;">— это возможность начать реальный технологический бизнес, не бросая учебу и не</span>
    </p><p style="margin:0;font-size:28px"><span style="font-size:28px;">выдумывая</span></p><p style="margin:0;font-size:28px"><span style="font-size:28px;">бизнес-идею.</span></p></div></div></td></tr></table></td><td class="column column-2" width="50%" style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;border-top:0;border-right:0;border-bottom:0;border-left:0"><table class="text_block" width="100%" border="0" cellpadding="0" cellspacing="0" 
    role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;word-break:break-word"><tr><td style="padding-bottom:10px;padding-left:5px;padding-right:5px;padding-top:10px"><div style="font-family:'Trebuchet MS',Tahoma,sans-serif"><div class="txtTinyMce-wrapper" style="font-size:14px;font-family:Montserrat,'Trebuchet MS','Lucida Grande','Lucida Sans Unicode','Lucida Sans',Tahoma,sans-serif;mso-line-height-alt:16.8px;color:#555;line-height:1.2"><p style="margin:0">
    <span style="color:#000000;">— Практиканты переезжают в Москву </span></p><p style="margin:0"><span style="color:#000000;">и в течение учебного года совмещают фулл-тайм работу в стартапе с дистационным обучением.</span></p><p style="margin:0;mso-line-height-alt:16.8px">&nbsp;</p><p style="margin:0"><span style="color:#000000;">— На практике студент занимает предпринимательскую позицию </span></p><p style="margin:0">
    <span style="color:#000000;">в одном из стартапов, открытых ТехноСпарком, и строит бизнес в его экосистеме.</span></p><p style="margin:0;mso-line-height-alt:16.8px">&nbsp;</p><p style="margin:0"><span style="color:#000000;">— ТехноСпарк обеспечивает практикантов официальным трудоустройством, заработной платой </span></p><p style="margin:0"><span style="color:#000000;">и инвестициями в стартап.</span></p><p style="margin:0;mso-line-height-alt:16.8px">&nbsp;</p><p style="margin:0">
    <span style="color:#000000;">— Университет помогает с учебным планом, а также оплачивает жилье</span></p><p style="margin:0"><span style="color:#000000;">и логистику.</span></p></div></div></td></tr></table></td></tr></tbody></table></td></tr></tbody></table><table class="row row-9" align="center" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff"><tbody><tr><td><table class="row-content stack" 
    align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff;color:#000;width:620px" width="620"><tbody><tr><td class="column column-1" width="100%" style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;padding-top:5px;padding-bottom:5px;border-top:0;border-right:0;border-bottom:0;border-left:0"><table class="button_block" width="100%" border="0" cellpadding="10" 
    cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0"><tr><td><div align="center">
    <!--[if mso]><v:roundrect xmlns:v="urn:schemas-microsoft-com:vml" xmlns:w="urn:schemas-microsoft-com:office:word" href="https://startupdiplom.ru/#AboutProgram" style="height:42px;width:251px;v-text-anchor:middle;" arcsize="10%" stroke="false" fillcolor="#db3a38"><w:anchorlock/><v:textbox inset="0px,0px,0px,0px"><center style="color:#ffffff; font-family:'Trebuchet MS', Tahoma, sans-serif; font-size:16px"><![endif]-->
    <a href="https://startupdiplom.ru/#AboutProgram" target="_blank" style="text-decoration:none;display:inline-block;color:#ffffff;background-color:#db3a38;border-radius:4px;width:auto;border-top:1px solid #db3a38;border-right:1px solid #db3a38;border-bottom:1px solid #db3a38;border-left:1px solid #db3a38;padding-top:5px;padding-bottom:5px;font-family:'Montserrat', 'Trebuchet MS', 'Lucida Grande', 'Lucida Sans Unicode', 'Lucida Sans', Tahoma, sans-serif;text-align:center;mso-border-alt:none;word-break:keep-all;"><span style="padding-left:20px;padding-right:20px;font-size:16px;display:inline-block;letter-spacing:normal;"><span style="font-size: 16px; line-height: 2; word-break: break-word; mso-line-height-alt: 32px;">Подробнее о Программе</span></span></a>
    <!--[if mso]></center></v:textbox></v:roundrect><![endif]--></div></td></tr></table></td></tr></tbody></table></td></tr></tbody></table><table class="row row-10" align="center" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff"><tbody><tr><td><table class="row-content stack" align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" 
    style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff;color:#333;width:620px" width="620"><tbody><tr><td class="column column-1" width="100%" style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;padding-top:30px;padding-bottom:30px;border-top:0;border-right:0;border-bottom:0;border-left:0"><table class="text_block" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" 
    style="mso-table-lspace:0;mso-table-rspace:0;word-break:break-word"><tr><td style="padding-left:15px;padding-right:15px;padding-top:40px"><div style="font-family:'Trebuchet MS',Tahoma,sans-serif"><div class="txtTinyMce-wrapper" style="font-size:12px;mso-line-height-alt:14.399999999999999px;color:#aaa;line-height:1.2;font-family:Montserrat,'Trebuchet MS','Lucida Grande','Lucida Sans Unicode','Lucida Sans',Tahoma,sans-serif"><p style="margin:0;font-size:20px">
    <span style="font-size:16px;color:#000000;"><span style="font-size:22px;"><span style="font-size:28px;color:#b91919;">ТехноСпарк</span> </span></span><span style="font-size:16px;color:#000000;"><span style="font-size:22px;">—</span> </span><span style="font-size:20px;"><span style="color:#000000;">э</span><span style="color:#000000;">то первая стартап-студия страны. Мы создаем технологические бизнесы в material based индустриях, таких как энергетика, фотовольтаика, логистика, робототехника и другие.</span></span>
    </p><p style="margin:0;font-size:20px;mso-line-height-alt:14.399999999999999px">&nbsp;</p><p style="margin:0;font-size:20px"><span style="font-size:20px;color:#000000;">ТехноСпарк серийно создает высокотехнологичные компании от идеи до продажи готового бизнеса, в&nbsp;том числе контрактные сервисы по производству, инжинирингу, промдизайну.</span></p><p style="margin:0;font-size:20px;mso-line-height-alt:14.399999999999999px">&nbsp;</p></div></div></td></tr></table></td></tr></tbody></table></td>
    </tr></tbody></table><table class="row row-11" align="center" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff"><tbody><tr><td><table class="row-content stack" align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff;color:#000;width:620px" width="620"><tbody><tr><td class="column column-1" width="100%" 
    style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;padding-top:5px;padding-bottom:5px;border-top:0;border-right:0;border-bottom:0;border-left:0"><table class="button_block" width="100%" border="0" cellpadding="10" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0"><tr><td><div align="center">
    <!--[if mso]><v:roundrect xmlns:v="urn:schemas-microsoft-com:vml" xmlns:w="urn:schemas-microsoft-com:office:word" href="https://technospark.ru" style="height:42px;width:263px;v-text-anchor:middle;" arcsize="10%" stroke="false" fillcolor="#db3a38"><w:anchorlock/><v:textbox inset="0px,0px,0px,0px"><center style="color:#ffffff; font-family:'Trebuchet MS', Tahoma, sans-serif; font-size:16px"><![endif]-->
    <a href="https://technospark.ru" target="_blank" style="text-decoration:none;display:inline-block;color:#ffffff;background-color:#db3a38;border-radius:4px;width:auto;border-top:1px solid #db3a38;border-right:1px solid #db3a38;border-bottom:1px solid #db3a38;border-left:1px solid #db3a38;padding-top:5px;padding-bottom:5px;font-family:'Montserrat', 'Trebuchet MS', 'Lucida Grande', 'Lucida Sans Unicode', 'Lucida Sans', Tahoma, sans-serif;text-align:center;mso-border-alt:none;word-break:keep-all;"><span style="padding-left:20px;padding-right:20px;font-size:16px;display:inline-block;letter-spacing:normal;"><span style="font-size: 16px; line-height: 2; word-break: break-word; mso-line-height-alt: 32px;">Подробнее о ТехноСпарке</span></span></a>
    <!--[if mso]></center></v:textbox></v:roundrect><![endif]--></div></td></tr></table></td></tr></tbody></table></td></tr></tbody></table><table class="row row-12" align="center" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff"><tbody><tr><td><table class="row-content stack" align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" 
    style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff;color:#000;width:620px" width="620"><tbody><tr><td class="column column-1" width="100%" style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;padding-top:5px;padding-bottom:5px;border-top:0;border-right:0;border-bottom:0;border-left:0"><table class="text_block" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" 
    style="mso-table-lspace:0;mso-table-rspace:0;word-break:break-word"><tr><td style="padding-bottom:10px;padding-left:10px;padding-right:10px;padding-top:35px"><div style="font-family:'Trebuchet MS',Tahoma,sans-serif"><div class="txtTinyMce-wrapper" style="font-size:14px;font-family:Montserrat,'Trebuchet MS','Lucida Grande','Lucida Sans Unicode','Lucida Sans',Tahoma,sans-serif;mso-line-height-alt:16.8px;color:#555;line-height:1.2"><p style="margin:0;font-size:14px;text-align:left">
    <span style="font-size:28px;color:#b91919;">В какой форме пройдет отбор в программу?</span></p></div></div></td></tr></table></td></tr></tbody></table></td></tr></tbody></table><table class="row row-13" align="center" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff"><tbody><tr><td><table class="row-content stack" align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" 
    style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff;color:#000;width:620px" width="620"><tbody><tr><td class="column column-1" width="100%" style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;padding-top:5px;padding-bottom:5px;border-top:0;border-right:0;border-bottom:0;border-left:0"><table class="text_block" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" 
    style="mso-table-lspace:0;mso-table-rspace:0;word-break:break-word"><tr><td style="padding-bottom:55px;padding-left:10px;padding-right:10px;padding-top:10px"><div style="font-family:'Trebuchet MS',Tahoma,sans-serif"><div class="txtTinyMce-wrapper" style="font-size:14px;font-family:Montserrat,'Trebuchet MS','Lucida Grande','Lucida Sans Unicode','Lucida Sans',Tahoma,sans-serif;mso-line-height-alt:16.8px;color:#555;line-height:1.2"><p style="margin:0;font-size:20px">
    <span style="font-size:20px;"><span style="color:#000000;">В программе всего <span style="color:#b91919;"><strong>3</strong></span> этапа:</span></span></p><p style="margin:0;font-size:20px;mso-line-height-alt:16.8px">&nbsp;</p><p style="margin:0;font-size:20px"><strong><span style="color:#b91919;">1 этап</span></strong></p><p style="margin:0;font-size:20px"><span style="color:#000000;"> Онлайн-практикум Школы предпринимательсва ТехноСпарк</span></p><p 
    style="margin:0;font-size:20px;mso-line-height-alt:16.8px">&nbsp;</p><p style="margin:0;font-size:20px"><strong><span style="font-size:20px;"><span style="color:#000000;"><span style="color:#b91919;">2 этап</span> </span></span></strong></p><p style="margin:0;font-size:20px"><span style="font-size:20px;"><span style="color:#000000;">Деловая игра «Построй компанию / Продай компанию». — ролевой симулятор технологической индустрии. </span></span></p><p style="margin:0;font-size:20px">
    <span style="font-size:20px;"><span style="color:#000000;">В основе ее сюжета — упорядоченный опыт массового строительства технологических компаний — как нашей сети, так и наших европейских партнеров, в первую очередь инновационных кластеров Лёвена </span></span></p><p style="margin:0;font-size:20px"><span style="font-size:20px;"><span style="color:#000000;">и Кембриджа.</span></span></p><p style="margin:0;font-size:20px">
    <span style="color:#000000;font-size:20px;">Попадая в игру, участники за короткое время проживают основные типовые бизнес-ситуации, характерные для реального технологического бизнеса.</span></p><p style="margin:0;font-size:20px;mso-line-height-alt:16.8px">&nbsp;</p><p style="margin:0;font-size:20px"><strong><span style="color:#b91919;font-size:20px;">3 этап</span></strong></p><p style="margin:0;font-size:20px">
    <span style="color:#000000;"><strong><span style="font-size:20px;">При успешном прохождении 1 и 2 этапа, тебя ждет приглашение на собеседование с экспертами Школы предпринимательства <span style="color:#b91919;">ТехноСпарк</span></span></strong></span></p></div></div></td></tr></table></td></tr></tbody></table></td></tr></tbody></table><table class="row row-14" align="center" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" 
    style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff"><tbody><tr><td><table class="row-content stack" align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff;color:#000;width:620px" width="620"><tbody><tr><td class="column column-1" width="50%" 
    style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;border-top:0;border-right:0;border-bottom:0;border-left:0"><table class="text_block" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;word-break:break-word"><tr><td style="padding-left:10px;padding-top:5px;padding-bottom:5px"><div style="font-family:'Trebuchet MS',Tahoma,sans-serif"><div class="txtTinyMce-wrapper" 
    style="font-size:14px;font-family:Montserrat,'Trebuchet MS','Lucida Grande','Lucida Sans Unicode','Lucida Sans',Tahoma,sans-serif;mso-line-height-alt:16.8px;color:#555;line-height:1.2"><p style="margin:0;font-size:18px"><span style="font-size:20px;color:#000000;">Деловая игра «Построй компанию / Продай компанию»</span></p><p style="margin:0;font-size:18px"><span style="font-size:18px;color:#000000;">— это не только отбор в программу предпринимательской практики, </span></p><p 
    style="margin:0;font-size:18px"><span style="font-size:18px;color:#000000;">но и шанс прокачать </span></p><p style="margin:0;font-size:18px"><span style="font-size:18px;color:#000000;">свои деловые </span></p><p style="margin:0;font-size:18px"><span style="font-size:18px;color:#000000;">навыки и научиться говорить </span></p><p style="margin:0;font-size:18px"><span style="font-size:18px;color:#000000;">на одном языке с главными технологическими компаниями мира.</span></p></div></div></td></tr>
    </table></td><td class="column column-2" width="50%" style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;border-top:0;border-right:0;border-bottom:0;border-left:0"><table class="text_block" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;word-break:break-word"><tr><td style="padding-top:5px;padding-bottom:5px"><div style="font-family:'Trebuchet MS',Tahoma,sans-serif"><div 
    class="txtTinyMce-wrapper" style="font-size:14px;font-family:Montserrat,'Trebuchet MS','Lucida Grande','Lucida Sans Unicode','Lucida Sans',Tahoma,sans-serif;mso-line-height-alt:16.8px;color:#555;line-height:1.2"><p style="margin:0;font-size:14px;letter-spacing:normal"><span style="font-size:28px;color:#b91919;">Лучшие участники&nbsp; попадут </span></p><p style="margin:0;font-size:14px;letter-spacing:normal"><span style="font-size:28px;color:#b91919;">на оплачиваемую практику </span></p><p 
    style="margin:0;font-size:14px;letter-spacing:normal"><span style="font-size:28px;color:#b91919;">в технологический стартап ТехноСпарка!</span></p></div></div></td></tr></table></td></tr></tbody></table></td></tr></tbody></table><table class="row row-15" align="center" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff"><tbody><tr><td><table class="row-content stack" align="center" border="0" 
    cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff;color:#000;width:620px" width="620"><tbody><tr><td class="column column-1" width="100%" style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;padding-top:5px;padding-bottom:5px;border-top:0;border-right:0;border-bottom:0;border-left:0"><table class="heading_block" width="100%" border="0" cellpadding="15" cellspacing="0" 
    role="presentation" style="mso-table-lspace:0;mso-table-rspace:0"><tr><td><h1 style="margin:0;color:#555;direction:ltr;font-family:Montserrat,'Trebuchet MS','Lucida Grande','Lucida Sans Unicode','Lucida Sans',Tahoma,sans-serif;font-size:23px;font-weight:700;letter-spacing:normal;line-height:120%;text-align:center;margin-top:0;margin-bottom:0"><span class="tinyMce-placeholder" style="color: #b91919;">Дату игры уточняй в своем университете!</span></h1></td></tr></table></td></tr></tbody></table>
    </td></tr></tbody></table><table class="row row-16" align="center" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff"><tbody><tr><td><table class="row-content stack" align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff;color:#000;width:620px" width="620"><tbody><tr><td class="column column-1" width="25%" 
    style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;border-top:0;border-right:0;border-bottom:0;border-left:0"><table class="image_block" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0"><tr><td style="padding-bottom:50px;padding-left:45px;padding-right:45px;padding-top:50px;width:100%"><div align="center" style="line-height:10px"><img 
    src="https://10education.ru/images/007-reminder.png" style="display:block;height:auto;border:0;width:65px;max-width:100%" width="65"></div></td></tr></table></td><td class="column column-2" width="75%" style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;border-top:0;border-right:0;border-bottom:0;border-left:0"><table 
    class="text_block" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;word-break:break-word"><tr><td style="padding-bottom:10px;padding-left:5px;padding-right:5px;padding-top:60px"><div style="font-family:'Trebuchet MS',Tahoma,sans-serif"><div class="txtTinyMce-wrapper" 
    style="font-size:14px;font-family:Montserrat,'Trebuchet MS','Lucida Grande','Lucida Sans Unicode','Lucida Sans',Tahoma,sans-serif;mso-line-height-alt:16.8px;color:#555;line-height:1.2"><p style="margin:0;font-size:14px"><span style="font-size:20px;color:#000000;">Мы будем отправлять информационные сообщения о следующих этапах.</span></p></div></div></td></tr></table></td></tr></tbody></table></td></tr></tbody></table><table class="row row-17" align="center" width="100%" border="0" 
    cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff"><tbody><tr><td><table class="row-content stack" align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#fff;color:#000;width:620px" width="620"><tbody><tr><td class="column column-1" width="100%" 
    style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;padding-top:5px;padding-bottom:5px;border-top:0;border-right:0;border-bottom:0;border-left:0"><table class="text_block" width="100%" border="0" cellpadding="10" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;word-break:break-word"><tr><td><div style="font-family:'Trebuchet MS',Tahoma,sans-serif"><div class="txtTinyMce-wrapper" 
    style="font-size:14px;font-family:Montserrat,'Trebuchet MS','Lucida Grande','Lucida Sans Unicode','Lucida Sans',Tahoma,sans-serif;mso-line-height-alt:16.8px;color:#555;line-height:1.2"><p style="margin:0;font-size:22px"><span style="color:#000000;">С уважением,</span></p><p style="margin:0;font-size:22px;mso-line-height-alt:16.8px">&nbsp;</p><p style="margin:0;font-size:22px"><span style="color:#000000;font-size:22px;">Команда Школы предпринимательства ТехноСпарка</span></p></div></div></td>
    </tr></table></td></tr></tbody></table></td></tr></tbody></table><table class="row row-18" align="center" width="100%" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#2d2d2d"><tbody><tr><td><table class="row-content stack" align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;background-color:#2d2d2d;color:#333;width:620px" width="620"><tbody><tr><td 
    class="column column-1" width="100%" style="mso-table-lspace:0;mso-table-rspace:0;font-weight:400;text-align:left;vertical-align:top;padding-top:25px;padding-bottom:25px;border-top:0;border-right:0;border-bottom:0;border-left:0"><table class="text_block" width="100%" border="0" cellpadding="10" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0;word-break:break-word"><tr><td><div style="font-family:'Trebuchet MS',Tahoma,sans-serif"><div class="txtTinyMce-wrapper" 
    style="font-size:14px;mso-line-height-alt:16.8px;color:#555;line-height:1.2;font-family:Montserrat,'Trebuchet MS','Lucida Grande','Lucida Sans Unicode','Lucida Sans',Tahoma,sans-serif"><p style="margin:0"><span style="color:#ffffff;">Остались сомнения или есть вопросы про программу или практикум? Напишите нам на почту, в телеграм или чат на сайте, ссылки указаны ниже. </span></p><p style="margin:0"><span style="color:#ffffff;">Мы расскажем больше, как всё устроено.</span></p></div></div></td>
    </tr></table><table class="social_block" width="100%" border="0" cellpadding="10" cellspacing="0" role="presentation" style="mso-table-lspace:0;mso-table-rspace:0"><tr><td><table class="social-table" width="111px" border="0" cellpadding="0" cellspacing="0" role="presentation" align="center" style="mso-table-lspace:0;mso-table-rspace:0"><tr><td style="padding:0 5px 0 0"><a href="mailto:game@edu.technospark.ru" target="_blank"><img 
    src="https://10education.ru/images/mail@2x.png" width="32" height="32" alt="E-Mail" title="E-Mail" style="display:block;height:auto;border:0"></a></td><td style="padding:0 5px 0 0"><a href="https://t.me/techpreneur_school" target="_blank"><img src="https://10education.ru/images/telegram@2x.png" width="32" height="32" alt="Telegram" title="Telegram" 
    style="display:block;height:auto;border:0"></a></td><td style="padding:0 5px 0 0"><a href="http://edu.technospark.ru" target="_blank"><img src="https://10education.ru/images/website@2x.png" width="32" height="32" alt="Web Site" title="Web Site" style="display:block;height:auto;border:0"></a></td></tr></table></td></tr></table></td></tr></tbody></table></td></tr></tbody></table></td></tr></tbody></table><!-- End --></body></html>
    """

    # construct email
    email_to = email
    email = EmailMessage()
    email['Subject'] = 'Технологическое предпринимательство: результаты практикума'
    email['From'] = 'game@edu.technospark.ru'
    email['To'] = email_to
    email.set_content(html, subtype='html')

    # Send the message via SMTP server.
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL('smtp.mail.ru', 465, context=context) as s:
        s.login(email['From'], "game2017ts")
        s.send_message(email)

In [4]:
sa = gspread.service_account()
sh = sa.open("practice_finish")

wks = sh.worksheet("finish")
finish = pd.DataFrame(wks.get_all_records())
finish['ma_email'] = finish['ma_email'].str.lower()

"""
for email in finish['ma_email'].unique()[:1]:
    if finish[finish['ma_email']==email]['Done'].min() != 'TRUE':
        print(email)
        send_finish(email)
        for index in finish[finish['ma_email']==email].index:
            wks.update_cell(index+2, 10, True)
"""

vik.mikov@yandex.ru


You should consider upgrading via the 'c:\users\андрей\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 103
Current browser version is 106.0.5249.121 with binary path C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	Ordinal0 [0x00C76463+2188387]
	Ordinal0 [0x00C0E461+1762401]
	Ordinal0 [0x00B23D78+802168]
	Ordinal0 [0x00B44E59+937561]
	Ordinal0 [0x00B40A6C+920172]
	Ordinal0 [0x00B3E331+910129]
	Ordinal0 [0x00B71430+1119280]
	Ordinal0 [0x00B7108A+1118346]
	Ordinal0 [0x00B6C5F6+1099254]
	Ordinal0 [0x00B46BE0+945120]
	Ordinal0 [0x00B47AD6+948950]
	GetHandleVerifier [0x00F171F2+2712546]
	GetHandleVerifier [0x00F0886D+2652765]
	GetHandleVerifier [0x00D0002A+520730]
	GetHandleVerifier [0x00CFEE06+516086]
	Ordinal0 [0x00C1468B+1787531]
	Ordinal0 [0x00C18E88+1805960]
	Ordinal0 [0x00C18F75+1806197]
	Ordinal0 [0x00C21DF1+1842673]
	BaseThreadInitThunk [0x74028494+36]
	RtlAreBitsSet [0x774841C8+136]
	RtlAreBitsSet [0x77484198+88]
